# DSE 241 - Final Project
# Rock & Fucking Roll

## Requirements

In [16]:
import requests
import pandas as pd
import numpy as np
import datetime

## Functions

In [17]:
def read_api_key(file_name):
    f = open(file_name, 'r')
    key = f.read()
    f.close()
    return key

def getgenre_artist(string):
    ## lower case
    ## replace " " with "_"
    artist = string.replace(' ','_').lower()
    return artist

def bands_artist(string):
    ## url-ify artist names
    ## replaces " " with "%20"
    artist = string.replace(' ',r'%20')
    return artist
    
    
def parse_events(event_json):
    ### defined to help parse the JSON of individual events returned by Bandintown API ###
    ### this will pull out the keys and values for the event object ###
    ### the end result is a list that can be entered as a row to a Pandas dataframe ###
    
    ### TODO: determine if the output is better in Pandas or Numpy ###
    ### TODO: define event_json object ###
    
    events_data = []
    num_of_events = len(event_json) # event json has an entry for each event ## each event has lots of other nested data

    keys_of_interest = [
            'datetime' # date-time of the event
            , 'title'  # name of the event  ## may need to create a condition to check if exists and set to null if missing
            , 'lineup' # list of strings containing names of artists  ## may leave as a nested list in data frame to avoid sparseness  ### list may be ordered in terms of headliners (??)
            , 'festival_start_date' # date festival starts  ### may be a useful indicator that the artist is performing at a festival-event, need to test this for reliability
            , 'festival_end_date'   # date festival ends, will differ from start date on multi-day events (may not exist on single day events)
            , 'venue'  # this is a nested dictionary, will need to tease this one out to flatten the data
    ]

    venue_keys_of_interest = [
             'city'        # city name, string
            , 'region'     # state-level, string
            , 'country'    # country name, string
            , 'latitude'   # coordinate data, float
            , 'longitude'  # coordinate data, float
            , 'location'   # arbitrary string describing geolocation  ## consider not including
            , 'name'       # arbitrary string describing venue name, could be misleading since some venues are at locations, but given festival name  ## consider not including
    ]

    for i in range(num_of_events):
        event = event_json[i]
        event_list = []

        for key in keys_of_interest:
            value = event.get(key)
            if key == 'venue':
                for venue_key in venue_keys_of_interest:
                    venue_value = value.get(venue_key)
                    event_list.append(venue_value)
            else:
                event_list.append(value)

        events_data.append(event_list)


    cols = keys_of_interest[:-1] + venue_keys_of_interest
    events_df = pd.DataFrame(events_data, columns=cols)
    events_df['artist'] = artist
    return events_df

def get_locations(df, latitude_column='latitude', longitude_column='longitude'):
    new_df = df.copy()
    geolocator = Nominatim(user_agent="rg_agent")
    reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    for index, row in new_df.iterrows():
        lookup = reverse_geocode((row[latitude_column],row[longitude_column]), language='en')
        try:
            new_df.loc[index,'city'] = lookup.raw['address']['city']
        except:
            continue
        try:
            new_df.loc[index,'region'] = lookup.raw['address']['state']
        except:
            continue
        try:
            new_df.loc[index,'country'] = lookup.raw['address']['country']
        except:
            continue

    return new_df

## Key Inputs

In [18]:
artist = 'Tame Impala'

bands_key = read_api_key('bands_api_key.txt') ## shhhh... its a secret

analysis_level = 1
gg_artist = getgenre_artist(artist)
bands_artist = bands_artist(artist)
getgenre_api_url = r'https://api.getgenre.com/search?artist_name={}&analysis={}'.format(gg_artist, analysis_level)
bands_api_url = r'https://rest.bandsintown.com/artists/{}/events/?app_id={}'.format(bands_artist, bands_key)
bands_api_url_past = r'https://rest.bandsintown.com/artists/{}/events/?app_id={}&date=past'.format(bands_artist, bands_key)

In [19]:
# gg_artist

In [20]:
# bands_artist

## Working Code
### Let's Go

In [21]:
# data extract
r = requests.get(bands_api_url_past)
event_json = r.json()

test_df = parse_events(event_json)
print(test_df.shape)
test_df.head()

(319, 13)


,datetime,title,lineup,festival_start_date,festival_end_date,city,region,country,latitude,longitude,location,name,artist
0,2013-04-14T10:08:00,,"[Tame Impala, The Airborne Toxic Event, Eric P...",,,Indio,CA,United States,33.7205556,-116.2147222,,Empire Polo Field,Tame Impala
1,2013-04-19T19:00:00,,"[Tame Impala, Mona, Alt-J, Social Distortion, ...",,,Indio,CA,United States,33.6784492,-116.237155,,Empire Polo Club,Tame Impala
2,2013-04-21T10:26:00,,"[Tame Impala, Vampire Weekend, Ladies Night, M...",,,Indio,CA,United States,33.7205556,-116.2147222,,Empire Polo Field,Tame Impala
3,2013-04-26T20:00:00,,"[Tame Impala, Midnight Juggernauts]",,,West Melbourne,,Australia,-37.81126,144.946442,,Festival Hall Melbourne,Tame Impala
4,2013-05-02T19:30:00,,[Tame Impala],,,Sydney,,Australia,-33.889832,151.223348,,Hordern Pavilion,Tame Impala


In [22]:
# data extract
r_gg = requests.get(getgenre_api_url)

genre_json = r_gg.json()
top_gg = genre_json['analysis']['top_genres']

In [23]:
# feature extraction
test_df['artist'] = artist
test_df['artist_topgenres'] = [top_gg for _ in range(len(test_df))]

test_df['lineup_size'] = test_df['lineup'].str.len()
test_df['festival_flag'] = np.where(test_df['lineup'].str.len() > 5, 'Festival', 'Concert')
test_df['festival_flag'] = test_df['festival_flag'].astype(str)

test_df['show_date'] = True

In [24]:
# data cleanup
print(test_df.shape)
test_df = test_df[(test_df['latitude'].notna()) & (test_df['longitude'].notna())]
print(test_df.shape)
test_df['latitude'] = pd.to_numeric(test_df['latitude'])
test_df['longitude'] = pd.to_numeric((test_df['longitude']))

test_df['datetime'] = test_df['datetime'].str[:10]

(319, 17)
(317, 17)


In [25]:
# data cleanup
#test_df = get_locations(test_df)  ## disabling for now due to run time

In [26]:
test_df.head()

,datetime,title,lineup,festival_start_date,festival_end_date,city,region,country,latitude,longitude,location,name,artist,artist_topgenres,lineup_size,festival_flag,show_date
0,2013-04-14,,"[Tame Impala, The Airborne Toxic Event, Eric P...",,,Indio,CA,United States,33.720556,-116.214722,,Empire Polo Field,Tame Impala,"[indie rock, psychedelic pop, psychedelic rock...",38,Festival,True
1,2013-04-19,,"[Tame Impala, Mona, Alt-J, Social Distortion, ...",,,Indio,CA,United States,33.678449,-116.237155,,Empire Polo Club,Tame Impala,"[indie rock, psychedelic pop, psychedelic rock...",95,Festival,True
2,2013-04-21,,"[Tame Impala, Vampire Weekend, Ladies Night, M...",,,Indio,CA,United States,33.720556,-116.214722,,Empire Polo Field,Tame Impala,"[indie rock, psychedelic pop, psychedelic rock...",42,Festival,True
3,2013-04-26,,"[Tame Impala, Midnight Juggernauts]",,,West Melbourne,,Australia,-37.811260,144.946442,,Festival Hall Melbourne,Tame Impala,"[indie rock, psychedelic pop, psychedelic rock...",2,Concert,True
4,2013-05-02,,[Tame Impala],,,Sydney,,Australia,-33.889832,151.223348,,Hordern Pavilion,Tame Impala,"[indie rock, psychedelic pop, psychedelic rock...",1,Concert,True


In [27]:
from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import get_provider, OSM #CARTODBPOSITRON_RETINA, WIKIMEDIA, ESRI_IMAGERY, STAMEN_TONER,
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, CustomJS
from bokeh.layouts import layout
from bokeh.models import DateRangeSlider
from pyproj import Transformer
#import xyzservices.providers as xyz

# data transformation
in_crs = 4326   # coordinates provided in EPSG:4326 format
out_crs = 3857   # coordinates output in EPSG 3857 (Web Mercator) format

transformer = Transformer.from_crs(in_crs, out_crs, always_xy=True)

lons, lats = [], []
for lon, lat in list(zip(test_df['longitude'], test_df['latitude'])):
    x, y = transformer.transform(lon,lat)
    lons.append(x)
    lats.append(y)

test_df['MercatorX'] = lons
test_df['MercatorY'] = lats



# visualization
output_file("tile.html")

tile_provider = get_provider(OSM)
tools = ['pan', 'zoom_in', 'zoom_out', 'wheel_zoom', 'box_zoom', 'lasso_select', 'tap', 'hover', 'reset', 'save']

tooltips = [
    ("Date", '@datetime'),
    ('Location: ', '@city, @region'),
    ('Country: ', '@country'),
    ('Event Type: ', '@festival_flag'),
    ('Lineup: ', '@lineup'),
]

source = ColumnDataSource(test_df)
source2 = ColumnDataSource(test_df)

show_dat = source.data['show_date']

view = CDSView(source=source, filters=[BooleanFilter(show_dat)])

date_start = test_df['datetime'].min()
date_end = test_df['datetime'].max()

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-18000000, 20000000), y_range=(-7500000, 11500000),
           #x_axis_type='mercator', y_axis_type='mercator',
           height=700, width=1500,
           tools=tools, tooltips=tooltips, active_scroll='wheel_zoom')
p.add_tile(tile_provider)
p.circle(x='MercatorX', y='MercatorY', size=6, fill_color='dodgerblue', line_color='dodgerblue', fill_alpha=.3, source=source, view=view)

# callback = CustomJS(args=dict(source=source, ref_source=source2), code="""
#     // print out array of date from, date to
#     console.log(cb_obj.value);
#
#     // dates returned from slider are not at round intervals and include time;
#     const date_from = Date.parse(new Date(cb_obj.value[0]).toDateString());
#     const date_to = Date.parse(new Date(cb_obj.value[1]).toDateString());
#     console.log(date_from, date_to)
#
#     // Creating the Data Sources
#     const data = source.data;
#     const ref = ref_source.data;
#
#     // Creating new Array and appending correctly parsed dates
#     let new_ref = []
#     ref["DateTime"].forEach(elem => {
#         elem = Date.parse(new Date(elem).toDateString());
#         new_ref.push(elem);
#         console.log(elem);
#     })
#
#     // Creating Indices with new Array
#     const from_pos = new_ref.indexOf(date_from);
#     const to_pos = new_ref.indexOf(date_to);
#
#
#     // re-create the source data from "reference"
#     data["datetime"] = ref["datetime"].slice(from_pos, to_pos);
#     source.change.emit();
#     """)
#
# date_slider = DateRangeSlider(
#     title=" Adjust Date range",
#     start=min(test_df['datetime']),
#     end=max(test_df['datetime']),
#     step=100,
#     value=(
#         min(test_df['datetime']), max(test_df['datetime'])
#     )
# )

#date_slider.js_link("value", p.x_range, "start", attr_selector=0)
#date_slider.js_link("value", p.x_range, "end", attr_selector=1)

#date_slider.js_on_change('value', callback)

# def update_plot(attr, old, new):
#      datesel = datetime.fromtimestamp(new / 1000).strftime('%Y-%m-%d')
#      new_data = test_df[test_df['date'] == datesel]
#      source.data.update(ColumnDataSource(new_data).data)
# date_slider.js_on_change('value', update_plot)

#https://data-dive.com/cologne-bike-rentals-interactive-map-bokeh-dynamic-choropleth
#https://stackoverflow.com/questions/55742220/converting-python-callback-function-to-javascript

# def update(source=source, slider=date_slider, window=None):
#     data=source.data
#     v_start = cb_obj.get('value')[0]
#     v_end = cb_obj.get('value')[1]
#     mask = np.logical_and(data['datetime'] > v_start, data['datetime'] < v_end)
#     source.data['show_date'] = mask
#     source.trigger('change')

#date_slider.js_on_change('value', CustomJS.from_py_func(update))

#layout = layout([date_slider], [p])
#show(layout)
show(p)

## Messy Workspace -- May not Run

In [28]:
source.data

{'index': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175,

In [29]:
source = {'data': {'index': [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 18,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68],
 'datetime': ['2016-12-03', '2017-01-20', '2017-04-22', '2017-04-23',
        '2017-07-22', '2017-08-20', '2017-09-15', '2017-12-29',
        '2018-03-16', '2018-04-22', '2018-06-01', '2018-06-02',
        '2018-06-03', '2018-06-10', '2018-06-29', '2018-07-21',
        '2018-09-09', '2018-09-29', '2018-10-05', '2018-10-26',
        '2018-10-27', '2018-11-02', '2018-11-10', '2018-11-17',
        '2018-12-29', '2018-12-31', '2019-03-01', '2019-05-04',
        '2019-05-10', '2019-05-11', '2019-05-18', '2019-05-19',
        '2019-06-07', '2019-06-08', '2019-06-23', '2019-07-18',
        '2019-07-19', '2019-07-20', '2019-07-21', '2019-08-01',
        '2019-09-20', '2019-09-23', '2019-10-11', '2020-03-06',
        '2020-07-17', '2020-10-08', '2021-05-19', '2021-06-18',
        '2021-07-10', '2021-07-16', '2021-07-22', '2021-07-29',
        '2021-07-30', '2021-08-06', '2021-08-12', '2021-08-13',
        '2021-08-14', '2021-08-27', '2021-09-10', '2021-10-02',
        '2021-10-07', '2021-10-08', '2021-10-15', '2021-10-16',
        '2021-11-27'],
 'title': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', ''],

 'lineup': [list(['Kerala Dust', 'Sentiment', 'Dwig', 'Kleintierschaukel', 'SNUR', 'Luke Redford']),
        list(['Kerala Dust', 'Harri', 'Kiosk', 'Constantijn Lange', 'Hobta', 'Jascha Hagen', 'Jonathan Lükermann', 'Chris Schwarzwälder']),
        list(['Kerala Dust', 'Monolink', 'Jacob Groening', 'Just Emma']),
        list(['Kerala Dust', 'Florian Rietze', 'Kiosk', 'Luca Musto', 'Viken Arman', 'YokoO', 'Stavroz', 'Coss', 'Genji Yoshida', 'Jacob Groening']),
        list(['Kerala Dust', 'Polo', 'Sainte Vie', 'Hyenah', 'Pan', 'Juan', 'Remcord', 'David Mears', 'Manumat', 'Raz Ohara', 'Sweet Melodic', 'Mark Höffen', 'Tomanka']),
        list(['Kerala Dust', 'Crussen', 'Armen Miran', 'Rampue', 'WooMoon', 'SPANIOL', 'Rico loop', 'Luca Musto']),
        list(['Kerala Dust']),
        list(['Kerala Dust', 'Marana', 'Javid', 'Elliver', 'Zigan Aldi', 'Karotte', 'Just Emma', 'The Sorry Entertainers', 'Moji ', 'Pauli Pocket', 'Oberst & Buchner', 'Britta Arnold', 'oberst']),
        list(['Kerala Dust', 'Betelgeize', 'Geju']),
        list(['Kerala Dust', 'More', 'Satori', 'Mira', 'Marty', 'Tickets', 'Francisco', 'Rega', 'Atish', 'Nico Stojan', 'Be Svendsen', 'Stavroz', 'Anthony Cardinale', 'Goga', 'Crussen', 'Ninze', 'Feathered Sun', 'Viken Arman', 'Oceanvs Orientalis', 'Sainte Vie', 'SPANIOL', 'David Paglia']),
        list(['Kerala Dust', 'wAFF']),
        list(['Kerala Dust', 'Stavroz', 'Goldcap', 'Viken Arman', 'Oceanvs Orientalis', 'Saand']),
        list(['Kerala Dust', 'Mira', 'Ramona Wouters']),
        list(['Kerala Dust', 'Crussen']),
        list(['Kerala Dust', 'Davi', 'Hraach', 'Sainte Vie', 'Timujin']),
        list(['Kerala Dust', 'Mutlu', 'Satori', 'Mira', 'Fame', 'Bernstein', 'Unders', 'Lovecraft', 'Bo', 'Peter Schumann', 'Sven Dohse', 'Britta Arnold', 'Be Svendsen', 'Jonathan Kaspar', 'Pauli Pocket', 'Daniel Cowel', 'mimi love', 'Caleesi', 'Timujin', 'Billy Caso', 'Sarah Kreis', 'Geju', 'borzu']),
        list(['Kerala Dust', 'Niko Schwind', 'Martha van Straaten', 'Sam Shure', 'Xique-Xique']),
        list(['Kerala Dust', 'Tristen', 'More Tba', 'Rampue', 'Ninze', 'Dwig', 'Die Wilde Jagd', 'Bad Stream']),
        list(['Kerala Dust']),
        list(['Kerala Dust', 'LCD Soundsystem', 'Yelle', 'Loco Dice', 'Apollonia', 'Shonky', 'Paco Osuna', 'Dan Ghenacia', 'Dyed Soundorom', 'Bakke', 'Ish', 'BLOND:ISH', 'Fat Nick', 'Peggy Gou']),
        list(['Kerala Dust', 'TrueVine']), list(['Kerala Dust']),
        list(['Kerala Dust', 'Roderic']),
        list(['Kerala Dust', 'Matanza', 'Isaac Soto']),
        list(['Kerala Dust', 'Pachanga Boys', 'Rodriguez Jr.', 'Axel Boman', 'Sandrino', 'Frankey', 'Concret']),
        list(['Kerala Dust', 'Robosonic']),
        list(['Kerala Dust', 'Zigan Aldi']),
        list(['Kerala Dust', 'Serge Devant', 'HUG', 'O', 'David Delgado', 'Shaun Reeves', 'DoubtingThomas', 'Peter Schumann', 'El Mundo', 'Detmolt', 'Just Emma', 'jPattersson', 'Love Over Entropy', 'UNDERHER', 'Niki Sadeki']),
        list(['Kerala Dust', 'Nico Stojan']),
        list(['Kerala Dust', 'Carlita', 'Red Axes', 'KMLN', 'Armen Miran', 'Gabriel Belmudes']),
        list(['Kerala Dust', 'Ramona Wouters']),
        list(['Kerala Dust', 'Lemurian', 'Lola Villa']),
        list(['Kerala Dust', 'Gorje Hewek', 'Izhevski', 'Secret Guest', 'Meandisco', 'KMLN', 'Veronika Fleyta', 'Geju', 'Sam Shure', 'Amonita']),
        list(['Kerala Dust', 'Stavroz', 'Crussen', 'PERSIAN EMPIRE']),
        list(['Kerala Dust', 'Serge Devant', 'Behrouz', 'Squire', 'Uone', 'Baiuca']),
        list(['Kerala Dust', 'Luciano', 'Very', 'Damian Lazarus', 'Rampue', 'Valentin Huedo', 'LUM', 'Black Motion', 'Jan Blomqvist', 'Huaira', 'Monolink', 'Sainte Vie']),
        list(['Kerala Dust', 'Robag Wruhme', 'MR', 'Kermesse', 'Ish', 'Lazare Hoche', 'Fatoumata', 'ALBA MUSIC', 'Hraach', 'Martha van Straaten', 'cantanca']),
        list(['Kerala Dust', 'Shimon', 'Kerri Chandler', 'Prins Thomas', 'Muallem', 'Recondite', 'Rampue', 'Stimming', 'Kiasmos', 'Christian Löffler', 'Seth Schwarz', 'Jan Blomqvist', 'Be Svendsen', 'Red Axes', 'Ben Rau', 'Konstantin Sibold', 'Parra for Cuva', 'Oceanvs Orientalis', 'Nicola Noir', 'Hraach', 'Emmanuel Russ', 'Fléoz', 'Paul Cut', 'Yarosslav', 'Tomanka']),
        list(['Kerala Dust', 'Shimon', 'Kerri Chandler', 'Rampue', 'Stimming', 'Christian Löffler', 'Seth Schwarz', 'Red Axes', 'Ben Rau', 'Nicola Noir', 'Paul Cut', 'Tomanka']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']),
        list(['Kerala Dust', 'Behrouz', 'Chaim', 'Oxia', 'Sabo', 'Raresh', 'Alex Niggemann', 'maher daniel', 'Nicolas Masseyeff', 'Bradley Zero', 'Praslea', 'Amine K', 'Zakaria', 'Konstantin Sibold', 'Two Faced', 'Marwan Sabb', 'Noritsu']),
        list(['Kerala Dust', 'Matija']),
        list(['Kerala Dust', 'Ruthless', 'Angerfist', 'Neophyte', 'DJ Coone', 'Jennifer Cardini', 'Mira', 'TNT', 'Âme', 'Poogie Bear', 'Shimon', 'Darren Styles', 'DJ Mad Dog', 'Tha Playah', 'Art of Fighters', 'The Viper', 'Dominik Eulberg', 'Evil Activities', 'Noize Suppressor', 'Endymion', 'The Prophet', 'Prins Thomas', 'Todd Terje', 'Headhunterz', 'Zatox', 'D-Block & S-Te-Fan', 'Korsakoff', 'Solstice', 'Partyraiser', 'N-Vitral', 'Very', 'Damian Lazarus', 'Batiste', 'Mystery', 'Resident DJ', 'Bass Modulators', 'AniMe', 'Noisecontrollers', 'Wildstylez', 'Da Tweekaz', 'Crypsis', 'Malaika', 'Dr. Evil', 'Dr. Rude', 'Dirty Bastards', 'Loc', 'BASSDRUM PROJECT', 'Broken Minds', 'Javi Boss', 'Frequencerz', 'Mano Le Tough', 'Act of rage', 'Demoniak', 'Javi Aznar', 'DJ Thomas', 'Warface', 'Vicente One More Time', 'Atmozfears', 'Dj Juanma', '&Me', 'Stavros', 'Dr. Phunk', 'Rico loop', 'Wakan', 'Wasted Penguinz', 'Dj DaY-mar', 'Jimi Jules', 'Chris Schwarzwälder', 'Radical Redemption', 'Stavroz', 'Destructive Tendencies', 'Abel K Kaña', 'Pastis & Buenri', 'Powel ', 'Javi Molina', 'Arzadous', 'Konstantin Sibold', 'Weval', 'Sub Zero Project', 'Quatri', 'Mira & Chris Schwarzwälder', 'M.RUX', 'Emmanuel Russ', 'Monolink', 'DIMA', 'HollSön', 'Lady Dammage', 'Tryangle Man', 'Billy Caso', 'Yarosslav', 'Deadly Guns', 'Greazy Puzzy Fuckerz', 'MC Tha Watcher', 'An Eagle In Your Mind']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust', 'Jennifer Cardini', 'Jimi Jules', 'Konstantin Sibold', 'Christopher Schwarzwälder', 'Viken Arman', 'Satori Musica']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust']), list(['Kerala Dust']),
        list(['Kerala Dust', 'Stavroz', 'Eagles & Butterflies', 'Perel'])],

 'festival_start_date': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', ''],

 'festival_end_date': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', ''],

 'city': ['Zurich', 'Berlin', 'Hamburg', 'Berlin', 'Paris', 'Ibiza', 'Miami',
        'Berlin', 'Moscow', 'Paris', 'Miami', 'Los Angeles', 'Denver',
        'San Francisco', 'Ibiza', 'Berlin', 'Berlin-Altglienicke',
        'Berlin', 'Munich', 'Miami', 'Miami', 'Montreal', 'Center',
        'Center', 'Quintana Roo', 'Quintana Roo', 'Berlin', 'Berlin',
        'San Francisco', 'Queens Village', 'Denver', 'Washington',
        'Moscow', 'London', 'Ibiza', 'Barcelona', 'Mugla',
        'VITRY SUR SEINE', 'VITRY SUR SEINE', 'Diepholz', 'Miami',
        'Los Angeles', 'Essaouira', 'Baden', 'Paris', 'Cologne', 'Hamburg',
        'Kyiv', 'Haale', 'Vitry-sur-Seine', 'Çeşme', 'Berlin', 'Berlin',
        'Mykonos', 'Chorzów', 'Kolkwitz', 'Zürich', 'Lärz',
        'Offenbach am Main', 'İstanbul', 'Basel', 'Zurich', 'Prague',
        'Amsterdam', 'London'],
 'region': ['', '', '', '', '', '', 'FL', '', '', '', 'FL', 'California', '',
        'CA', '', '', '', '', '', 'FL', 'FL', 'QC', '', '', '', '', '', '',
        'CA', 'New York', 'CO', 'DC', '', '', '', '', '', '', '', '', 'FL',
        'CA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', ''],
 'country': ['Switzerland', 'Germany', 'Germany', 'Germany', 'France', 'Spain',
        'United States', 'Germany', 'Russia', 'France', 'United States',
        'United States', 'United States', 'United States', 'Spain',
        'Germany', 'Germany', 'Germany', 'Germany', 'United States',
        'United States', 'Canada', 'Mexico', 'Mexico', 'Mexico', 'Mexico',
        'Germany', 'Germany', 'United States', 'United States',
        'United States', 'United States', 'Russian Federation',
        'United Kingdom', 'Spain', 'Spain', 'Turkey', 'France', 'France',
        'Germany', 'United States', 'United States', 'Morocco',
        'Switzerland', 'France', 'Germany', 'Germany', 'Ukraine',
        'Germany', 'France', 'Turkey', 'Germany', 'Germany', 'Greece',
        'Poland', 'Germany', 'Switzerland', 'Germany', 'Germany', 'Turkey',
        'Switzerland', 'Switzerland', 'Czech Republic', 'Netherlands',
        'United Kingdom'],
 'latitude': [47.38151  , 52.511612 , 53.551802 , 52.511612 , 48.8741   ,
        38.894431 , 25.784476 , 52.511612 , 55.75533  , 48.86405  ,
        25.7844757, 34.0522222, 40.8661111, 37.749454 , 38.8841997,
        52.511612 , 52.4166667, 52.502522 , 48.15     , 25.7844757,
        25.7844757, 45.532771 , 23.9090939, 23.9090939, 20.8666667,
        20.8666667, 52.51744  , 52.511612 , 37.7688759, 40.7266667,
        39.7646511, 38.916281 , 55.7522222, 51.52374  , 38.90625  ,
        41.3688085, 37.2163889, 48.7890892, 48.7890892, 52.6      ,
        25.784673 , 34.0905773, 31.4696583, 47.4781575, 48.85341  ,
        50.951817 , 53.5564221, 50.4590832, 54.18333  , 48.78716  ,
        38.2329489, 52.52437  , 52.52437  , 37.44529  , 49.90304  ,
        51.75     , 47.343558 , 53.3051075, 50.1055151, 41.0660865,
        47.577376 , 47.3882217, 50.108181 , 52.3867989, 51.51962  ],
 'longitude': [ 8.53688000e+00,  1.34268700e+01,  9.95820900e+00,  1.34268700e+01,
         2.26298000e+00,  1.33146700e+00, -8.01931360e+01,  1.34268700e+01,
         3.76098100e+01,  2.33105000e+00, -8.01931356e+01, -1.18242778e+02,
        -8.60775000e+01, -1.22386009e+02,  1.40489600e+00,  1.34268700e+01,
         1.35500000e+01,  1.34659300e+01,  1.15833333e+01, -8.01931356e+01,
        -8.01931356e+01, -7.36202940e+01, -1.02633400e+02, -1.02633400e+02,
        -8.86333333e+01, -8.86333333e+01,  1.34189900e+01,  1.34268700e+01,
        -1.22419427e+02, -7.37419444e+01, -1.04979783e+02, -7.70213690e+01,
         3.76155556e+01, -7.88390000e-02,  1.41953000e+00,  2.14827320e+00,
         2.83638889e+01,  2.38828990e+00,  2.38828990e+00,  8.36666670e+00,
        -8.01932020e+01, -1.18312565e+02, -9.76738950e+00,  8.30865560e+00,
         2.34880000e+00,  6.91709700e+00,  9.96997760e+00,  3.06378697e+01,
         9.55000000e+00,  2.40332000e+00,  2.63675087e+01,  1.34105300e+01,
         1.34105300e+01,  2.53287200e+01,  2.25633400e+01,  1.42500000e+01,
         8.53582910e+00,  1.27514235e+01,  8.73469550e+00,  2.90174246e+01,
         7.58632300e+00,  8.51947020e+00,  1.44431801e+01,  4.86998990e+00,
        -1.02412000e-01],
 'location': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 'name': ['Kauz', 'Kater Blau', 'Gruenspan', 'Kater Blau',
        'Les Pavillons des Etangs', 'Cova Santa', 'FLOYD', 'Kater Blau',
        'Leveldva', 'TBA', 'Club Space', 'TBA', 'TBA', 'The Midway',
        'Ushuaïa Ibiza Beach Hotel', 'Kater Blau', 'Sage Beach Berlin',
        '://about blank', 'The Drunken Monkey', 'Club Space',
        'Floyd Miami', 'Bar Le Ritz PDB', 'Proyecto Pblico Prim',
        'Sirilo Instabar', 'AAK AL Sacbe Lagoon', 'Gitano Tulum',
        'Beate Uwe', 'Kater Blau', 'Public Works', 'Knockdown Center',
        'Walnut Room', 'Flash', 'Community Moscow', 'Village Underground',
        'Cova Santa', 'Poble Espanyol', 'Xuma Yalikavak', 'Le Kilowatt',
        'Le Kilowatt', 'Bürgerpark (appletree garden)', 'The Ground - FL',
        'Gold Diggers', 'Hotel Sofitel Essaouira Mogador Golf & Spa',
        'Royal', 'Le Kilowatt', 'Club Bahnhof Ehrenfeld',
        'Uebel & Gefährlich', 'Kerala Dust @ White Night Festival',
        'Utopia Festival 2021', 'RITUEL',
        'Kerala Dust live at Sommer Klein',
        'Kerala Dust - live in concert', 'Kerala Dust Live in Concert',
        'Kerala Dust live @ Scorpios Mykonos', 'Fest Festival 2021',
        'Akasha Festival 2021', 'Lethargy Zurich 2021',
        'Kerala Dust live at Plan:et C', 'Kerala Dust Live at Sommerbnau',
        'Kerala Dust at Sonar Istanbul', 'Kerala Dust at Gannet',
        'Kerala Dust at Exil', 'Kerala Dust live in Prague',
        'The Gardens of Babylon @ ADE 2021', 'Fabric'],
 'artist': ['Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust', 'Kerala Dust', 'Kerala Dust', 'Kerala Dust',
        'Kerala Dust'],
 'artist_topgenres': [list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house']), list(['house']), list(['house']), list(['house']),
        list(['house'])],
 'lineup_size': [ 6,  8,  4, 10, 13,  8,  1, 13,  3, 22,  2,  6,  3,  2,  5, 23,  5,
         8,  1, 14,  2,  1,  2,  3,  7,  2,  2, 15,  2,  6,  2,  3, 10,  4,
         6, 12, 11, 25, 12,  1,  1,  1, 17,  2, 94,  1,  1,  1,  1,  7,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4],
 'festival_flag':['Festival', 'Festival', 'Concert', 'Festival', 'Festival',
        'Festival', 'Concert', 'Festival', 'Concert', 'Festival',
        'Concert', 'Festival', 'Concert', 'Concert', 'Concert', 'Festival',
        'Concert', 'Festival', 'Concert', 'Festival', 'Concert', 'Concert',
        'Concert', 'Concert', 'Festival', 'Concert', 'Concert', 'Festival',
        'Concert', 'Festival', 'Concert', 'Concert', 'Festival', 'Concert',
        'Festival', 'Festival', 'Festival', 'Festival', 'Festival',
        'Concert', 'Concert', 'Concert', 'Festival', 'Concert', 'Festival',
        'Concert', 'Concert', 'Concert', 'Concert', 'Festival', 'Concert',
        'Concert', 'Concert', 'Concert', 'Concert', 'Concert', 'Concert',
        'Concert', 'Concert', 'Concert', 'Concert', 'Concert', 'Concert',
        'Concert', 'Concert'],
 'MercatorX': [ 9.50321135e+05,  1.49467233e+06,  1.10854276e+06,  1.49467233e+06,
         2.51913781e+05,  1.48218228e+05, -8.92705906e+06,  1.49467233e+06,
         4.18670490e+06,  2.59491299e+05, -8.92705902e+06, -1.31627258e+07,
        -9.58210347e+06, -1.36239482e+07,  1.56392307e+05,  1.49467233e+06,
         1.50837910e+06,  1.49902047e+06,  1.28945076e+06, -8.92705902e+06,
        -8.92705902e+06, -8.19537364e+06, -1.14250978e+07, -1.14250978e+07,
        -9.86661753e+06, -9.86661753e+06,  1.49379513e+06,  1.49467233e+06,
        -1.36276683e+07, -8.20891570e+06, -1.16862960e+07, -8.57397958e+06,
         4.18734450e+06, -8.77631733e+03,  1.58021357e+05,  2.39144679e+05,
         3.15745367e+06,  2.65863216e+05,  2.65863216e+05,  9.31373077e+05,
        -8.92706641e+06, -1.31704945e+07, -1.08730083e+06,  9.24915311e+05,
         2.61467220e+05,  7.70007716e+05,  1.10985283e+06,  3.41059205e+06,
         1.06310114e+06,  2.67536359e+05,  2.93521764e+06,  1.49285337e+06,
         1.49285337e+06,  2.81958021e+06,  2.51173952e+06,  1.58630274e+06,
         9.50204149e+05,  1.41948197e+06,  9.72341855e+05,  3.23020493e+06,
         8.44505613e+05,  9.48383084e+05,  1.60780745e+06,  5.42124796e+05,
        -1.14004517e+04],
 'MercatorY': [6004570.06959557, 6893165.40093768, 7085725.81047338,
        6893165.40093768, 6253525.73562329, 4706560.98067053,
        2972411.76736251, 6893165.40093768, 7509862.16585027,
        6251824.92731495, 2972411.73027401, 4035816.34901808,
        4992613.09476467, 4544092.276759  , 4705097.72104785,
        6893165.40093768, 6875817.62675196, 6891502.91573991,
        6131845.86803904, 2972411.73027401, 2972411.73027401,
        5705788.98872309, 2742334.71139851, 2742334.71139851,
        2375987.0990246 , 2375987.0990246 , 6894231.47445908,
        6893165.40093768, 4546826.98320842, 4972108.27822514,
        4831800.72686635, 4709686.62900351, 7509247.401857  ,
        6714465.43048063, 4708251.57233031, 5066893.97798572,
        4469311.68236285, 6239149.72266987, 6239149.72266987,
        6909348.78733988, 2972436.12216593, 4040970.83862009,
        3693894.60345145, 6020473.79844093, 6250024.6427334 ,
        6612775.12937624, 7086591.55517114, 6526163.85395143,
        7204953.54127515, 6238823.7621892 , 4612386.14745461,
        6895499.31286198, 6895499.31286198, 4501357.56115723,
        6429500.97888261, 6755047.86303445, 5998332.8910321 ,
        7039635.1735204 , 6464569.28977139, 5022094.29977453,
        6036831.05642938, 6005673.56348724, 6465032.00585329,
        6870368.1055111 , 6713728.33196985],
 'show_date': [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]}}

def update(source=source, slider=date_slider, window=None):
    data=source.data
    v_start = cb_obj.get('value')[0]
    v_end = cb_obj.get('value')[1]
    mask = np.logical_and(data['datetime'] > v_start, data['datetime'] < v_end)
    source.data['show_date'] = mask
    source.trigger('change')

NameError: name 'date_slider' is not defined

In [ ]:
ColumnDataSource(test_df[(test_df['datetime'] > "2017-01-01") & (test_df['datetime'] < "2018-01-01")]).data

In [ ]:
mask = np.logical_and(source.data['datetime'] > '2017-07-22', source.data['datetime'] < '2018-01-01')
col = source.data['datetime']
col[mask] = 'replacement'
col

In [ ]:
''' A crossfilter plot map that uses the `Auto MPG dataset`_. This example
demonstrates the relationship of datasets together. A hover tooltip displays
information on each dot.
.. note::
    This example needs the Pandas package to run.
.. _Auto MPG dataset: https://archive.ics.uci.edu/ml/datasets/auto+mpg
'''
import pandas as pd

from bokeh.layouts import column, row
from bokeh.models import Select
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg_clean as df

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

widths = [7, 4, 10, 10, 11, 7, 4, 4, 30]

df = pd.read_fwf(url, widths=widths, header=None, na_values=['?'])
df.columns = ['mpg','cylinders','displacement','horsepower','weight','acceleration','model year','origin','car name']

SIZES = list(range(6, 22, 3))
COLORS = Spectral5
N_SIZES = len(SIZES)
N_COLORS = len(COLORS)

# data cleanup
df['cylinders'] = df['cylinders'].astype(str)
df['model year'] = df['model year'].astype(str)
del df['car name']

columns = sorted(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
continuous = [x for x in columns if x not in discrete]

def create_figure():
    xs = df[x.value].values
    ys = df[y.value].values
    x_title = x.value.title()
    y_title = y.value.title()

    kw = dict()
    if x.value in discrete:
        kw['x_range'] = sorted(set(xs))
    if y.value in discrete:
        kw['y_range'] = sorted(set(ys))
    kw['title'] = "%s vs %s" % (x_title, y_title)

    p = figure(height=600, width=800, tools='pan,box_zoom,hover,reset', **kw)
    p.xaxis.axis_label = x_title
    p.yaxis.axis_label = y_title

    if x.value in discrete:
        p.xaxis.major_label_orientation = pd.np.pi / 4

    sz = 9
    if size.value != 'None':
        if len(set(df[size.value])) > N_SIZES:
            groups = pd.qcut(df[size.value].values, N_SIZES, duplicates='drop')
        else:
            groups = pd.Categorical(df[size.value])
        sz = [SIZES[xx] for xx in groups.codes]

    c = "#31AADE"
    if color.value != 'None':
        if len(set(df[color.value])) > N_COLORS:
            groups = pd.qcut(df[color.value].values, N_COLORS, duplicates='drop')
        else:
            groups = pd.Categorical(df[color.value])
        c = [COLORS[xx] for xx in groups.codes]

    p.circle(x=xs, y=ys, color=c, size=sz, line_color="white", alpha=0.6, hover_color='white', hover_alpha=0.5)

    return p


def update(attr, old, new):
    layout.children[1] = create_figure()


x = Select(title='X-Axis', value='mpg', options=columns)
x.on_change('value', update)

y = Select(title='Y-Axis', value='horsepower', options=columns)
y.on_change('value', update)

size = Select(title='Size', value='None', options=['None'] + continuous)
size.on_change('value', update)

color = Select(title='Color', value='None', options=['None'] + continuous)
color.on_change('value', update)

controls = column(x, y, color, size, width=200)
layout = row(controls, create_figure())

show(layout)

# curdoc().add_root(layout)
# curdoc().title = "Crossfilter"
# curdoc()

In [ ]:
bokeh serve --show crossfilter

In [ ]:

df.head()

## Also Messy Workspace -- May not Run

In [ ]:
from pyproj import Transformer

in_crs = 4326   # coordinates provided in EPSG:4326 format
out_crs = 3857   # coordinates output in EPSG 3857 (Web Mercator) format

transformer = Transformer.from_crs(in_crs, out_crs, always_xy=True)

world_lon1, world_lat1 = transformer.transform(-180,-85)
world_lon2, world_lat2 = transformer.transform(180,85)

starbucks = pd.read_csv("directory.csv")
starbucks.head()
starbucks_us = starbucks[starbucks.Country == "US"].copy()

lons, lats = [], []
for lon, lat in list(zip(starbucks_us["Longitude"], starbucks_us["Latitude"])):
    x, y = transformer.transform(lon,lat)
    lons.append(x)
    lats.append(y)

starbucks_us["MercatorX"] = lons
starbucks_us["MercatorY"] = lats

starbucks_us = starbucks_us.rename(columns={"Store Name":"Name", "State/Province":"State"})
starbucks_us.head()

In [ ]:
## TODO add interactive buttons to switch between artists / other slices
## https://towardsdatascience.com/visualization-with-plotly-express-comprehensive-guide-eb5ee4b50b57#f38e
## https://plotly.com/python/dropdowns/

In [ ]:
# visualize data
import plotly.express as px
px.set_mapbox_access_token(open("mapbox_token.txt").read())
fig = px.scatter_mapbox(test_df,
                     lat='latitude',
                     lon='longitude',
                     color="festival_flag", # which column to use to set the color of markers
                     color_discrete_sequence=px.colors.qualitative.Vivid,

                     hover_name="city", # column added to hover information
                     hover_data={'datetime': True,
                                 'festival_flag': True,
                                 'latitude': False,
                                 'longitude': False},
                     labels={'festival_flag': 'Event Type',
                             'datetime': 'Date'},


                     #size="lineup_size", # size of markers
                     #projection="natural earth"
                     mapbox_style="streets",
                     zoom=1
                    )

# fig.update_layout(
#     mapbox_style="streets",
#     # mapbox_layers=[
#     #     {
#     #         "below": 'traces',
#     #         "sourcetype": "raster",
#     #         "sourceattribution": "United States Geological Survey",
#     #         "source": [
#     #             "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
#     #         ]
#     #     }
#     #   ]
# )
#fig.update_traces(hovertemplate= "%{label}: <br>Popularity: %{value} </br> %{text}")

# fig.update_layout(
#     margin={"r":0,"t":0,"l":0,"b":0},
#     updatemenus=[
#         dict(
#             buttons=list([
#                 dict(
#                     args=["marker.color",["red"]],
#                     label='Red',
#                     method='restyle',
#                     visible=True
#                 )
#             ]),
#             direction="down",
#             pad={'r': 10, 't': 10},
#             #showactive=True
#         )
#     ]
# )

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'customdata':[['2024-04-19', 'Festival', 33.6784492, -116.237155]]}],
                    label='Custom',
                    method='restyle',
                    visible=True
                )
            ]),
            direction="down",
            pad={'r': 10, 't': 10},
            #showactive=True
        )
    ]
)

fig.show()

In [ ]:
fig.to_dict()

In [ ]:
fig.data

In [ ]:
import reverse_geocode as rg
coordinates = (-37.81, 144.96), (31.76, 35.21)
loc = rg.search(coordinates)

In [ ]:
from functools import partial
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="rg_agent")

geocode = partial(geolocator.geocode, language="es")
print(geocode("london"))

print(geocode("paris"))

print(geocode("paris", language="en"))


reverse = partial(geolocator.reverse, language="es")
print(reverse("52.509669, 13.376294"))


In [ ]:
from geopy.extra.rate_limiter import RateLimiter
gr = RateLimiter(geolocator.reverse, min_delay_seconds=1)

x = gr((-37.81, 144.96),)
dir(x)

In [ ]:
x

In [ ]:
x.raw['address']['city']

In [ ]:
import plotly.express as px
#df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(test_df,
                     lat='latitude',
                     lon='longitude',
                     color="festival_flag", # which column to use to set the color of markers
                     hover_name="city", # column added to hover information
                     #size="lineup_size", # size of markers
                     projection="natural earth")
fig.update_geos(
    visible=False,
    resolution=50,
    #scope="north america",
    showcountries=True, countrycolor="Black"
    #,showsubunits=True, subunitcolor="Blue"
)
fig.show()

In [ ]:
x_df = test_df[:10].copy()
tt = get_location(x_df, 'latitude', 'longitude')
tt.head(10)

In [ ]:
test_df.head(10)

In [ ]:
test_df['city'] = test_df[['latitude', 'longitude']]

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
df['location'] = df['name'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
geolocator.reverse(-37.81, 144.96)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattergeo(
    test_df,
    lat='latitude',
    lon='longitude',
    color="festival_flag", # which column to use to set the color of markers
    hover_name="city", # column added to hover information
    #size="lineup_size", # size of markers
    projection="natural earth")
)
fig.update_geos(
    visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="lightGrey"
)
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import pandas as pd
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")
us_cities.head()

import plotly.express as px

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="City", hover_data=["State", "Population"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(test_df, lat="latitude", lon="longitude", hover_name="location",
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
ej = r.json()
ej[2]

In [ ]:
len(event_json)

In [ ]:
#r.text

In [ ]:
len(ej[0])

In [ ]:
ej[0].keys()

In [ ]:
list(ej[0].keys())

In [ ]:
type(ej[0])

In [ ]:
ej[0].values()

In [ ]:
v = ej[0].get('venue')
print(v)
type(v)

In [ ]:
v.get('country')

In [ ]:
for key in list(ej[0].keys()):
    print(key, " : \n", ej[0].get(key), "\n")